# YOLOv3 Crater Detection Algorithm Workflow
---
### Giles Matthews

In [ ]:
# GPU information
!nvidia-smi

## Setup

Clone the GitHub repo and install the requirements.txt file.

Having done this you are usually prompted to restart the runtime - do this using the prompt provided or by following:

    taskbar > Runtime > Restart runtime > Yes

Or use ```Ctrl + M .```

Re-import the necessary modules every runtime by running the appropriate cell below.

To access the datasets, please ensure this google drive folder is mounted:

https://drive.google.com/drive/folders/1Qh7VHt_dTZc8v0tYdSZlXo3cwTmxnYCI?usp=sharing

In [ ]:
print("---  Cloning repository  ---\n")
! git clone https://github.com/acse-gcm20/YOLOv3-CDA
print("\n---  Installing dependencies  ---\n")
! pip install -r /content/YOLOv3-CDA/requirements.txt
print("\nNow restart the Runtime before proceeding")

In [ ]:
# Import modules
# Run this cell every runtime
%cd /content/YOLOv3-CDA

import os
import torch
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np
from sklearn.metrics import auc
from PIL import Image
import random
from tqdm import tqdm

from pytorchyolo.train import run
from pytorchyolo.test import test
from pytorchyolo.detect import detect_directory

from src.utils import *

plt.rcParams.update({'font.size': 11})

/content/YOLOv3-CDA


In [ ]:
! git pull

remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (1/1), done.
remote: Total 4 (delta 3), reused 4 (delta 3), pack-reused 0
Unpacking objects: 100% (4/4), done.
From https://github.com/acse-gcm20/YOLOv3-CDA
   b30e82ce..07f41c30  master     -> origin/master
Updating b30e82ce..07f41c30
Fast-forward
 src/utils.py | 4 ++--
 1 file changed, 2 insertions(+), 2 deletions(-)


## Data Organisation

Datasets are stored in zip files in the Google Drive. To access these, unzip the files into the Colab runtime environment.

In [ ]:
# Extract Benedix data from google drive
%%capture
! unzip /content/drive/MyDrive/CDA/Benedix.zip -d data/Benedix

In [ ]:
# Extract Robbins data from google drive
%%capture
! unzip /content/drive/MyDrive/CDA/processed_Robbins.zip -d data/Robbins

In [ ]:
# Perform augmentation on Robbins data
# Takes ~7 mins
! python data/Robbins/crater_group_dataset/aug.py --dataset 'data/Robbins/crater_group_dataset' --id

imgs = os.listdir('data/Robbins/crater_group_dataset/images')
lbls = os.listdir('data/Robbins/crater_group_dataset/labels')
print(f'{len(imgs)} images, {len(lbls)} labels')

100% 3040/3040 [05:44<00:00,  8.82it/s]
12160 images, 12160 labels


## Training

Training, validation and testing path files for the Benedix, Robbins and Combined datasets are available in the 'data' directory.

Please edit the model configuration file (CDA.data) in the 'config' directory to choose which dataset to use. The default is to use the combined dataset.

Pretrained weights files are available in the 'weights' directory of the google drive.

In [ ]:
# Weights pretrained on COCO dataset
coco = '/content/drive/MyDrive/CDA/weights/coco.weights'

# Run training sequence
run('models/CDA.cfg', 50, 'config/CDA.data', pretrained_weights=coco)

In [ ]:
# Copy weights file to google drive
!cp checkpoints/yolov3_ckpt_50.pth /content/drive/MyDrive/CDA/weights/yolov3_ckpt_50.pth

In [ ]:
# Copy stats file to google drive
!cp stats.txt /content/drive/MyDrive/CDA/stats/stats.txt

## Analysis and Plotting

In [ ]:
# Plot the statistics generated during training.
plot_stats('stats.txt')

In [ ]:
# Plot a Precision-Recall curve to investigate performance across a range of confidence thresholds
set1 = PRcurve('models/CDA.cfg', 'checkpoints/yolov3_ckpt_50.pth', 'data/combined_valid.txt')
set1.save_stats('PRstats.txt')
set1.plot()

## Detections

Use a model and weights file to perfrom inference on a directory of images.

In [ ]:
# Move sample images from Benedix dataset to samples folder
list_file_2_dir('data/samples/samples.txt', 'data/samples/')

10 images
10 labels


In [ ]:
directory = '/content/YOLOv3-CDA/data/samples/images'
weights = 'checkpoints/yolov3_ckpt_50.pth'

detect_directory('models/CDA.cfg', weights, directory, ['crater'], 'output', batch_size=1, n_cpu=2, conf_thres=0.2)

In [ ]:
# Plot a comparison of a random sample of the ground-truth set with the detected labels.
comparison_plot('data/samples/images', 'data/samples/labels', 'output/labels', 4)

## Testing

In [ ]:
# Testing weights file on test set
test('models/CDA.cfg', 'checkpoints/yolov3_ckpt_1.pth', '/content/YOLOv3-CDA/config/CDA.data', '/content/YOLOv3-CDA/data/combined_test.txt')